In [1]:
import os
import pandas as pd
import csv

def read_files(path):
    file_list = []
    with os.scandir(path) as entries:
        for entry in entries:
            if entry.is_file():
                file_list.append(entry.name)
    return file_list

def clean_file_name(file):
    # 去除空格
    label_name = file[6:].replace(' ', '')
    # 去除后缀
    label_name = label_name[:-4]
    # 下划线转换成空格
    label_name = label_name.replace('_', ' ')
    return label_name

def read_folder(path):
    file_list = read_files(path)
    data = []
    for file in file_list:
        label_name = clean_file_name(file)
        file_path = os.path.join(path, file)
        with open(file_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                line = line.strip('\n').split(' ')
                data.append({
                    'ImageID': line[0],
                    'LabelName': label_name,
                    'X1': line[2],
                    'Y1': line[3],
                    'X2': line[4],
                    'Y2': line[5],
                    'X3': line[6],
                    'Y3': line[7],
                    'X4': line[8],
                    'Y4': line[9],
                    'Conf': line[1]
                })
    df = pd.DataFrame(data)
    return df

def write_csv(df, path):
    with open(path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(df.columns)
        writer.writerows(df.values)

def main():
    path = './943_hh/'
    suffix = '.bmp'
    topath = './submits/submit_943_hh.csv'
    df = read_folder(path)
    # df按LabelName排序（字典序从小到大），相同LabelName的按ImageID排序（转成数字类型，从小到大），如果前两项都相同，按Conf排序（从大到小）
    # ImageID转成数字类型，从小到大
    df['ImageID'] = df['ImageID'].astype(int)
    df = df.sort_values(by=['LabelName', 'ImageID', 'Conf'], ascending=[True, True, False])
    df['ImageID'] = df['ImageID'].astype(str)
    df['ImageID'] = df['ImageID'].apply(lambda x: x + suffix)
    write_csv(df, topath)

if __name__ == '__main__':
    main()


In [ ]:
# 读取csv文件为dataframe
df_copy = pd.read_csv('./submit_682.csv')
# 删除Conf列中小于0.6的行
df_copy = df_copy[df_copy['Conf'] >= 0.6]

# 导出csv文件
df_copy.to_csv('./submit_691.csv', index=False)